In [ ]:
import pandas as pd
from rdkit import Chem
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp

from tqdm.notebook import tqdm

from tqdm.contrib.concurrent import process_map

In [ ]:
data = pd.read_csv("./data_files/smiles_min_dist_natoms.csv",index_col=0)

In [ ]:
drugs = Chem.SDMolSupplier("./drugbank_structures.sdf")
drugs = [i for i in drugs if i]
for d in drugs:
    Chem.SanitizeMol(d)
    
    # Kekulization removed.
#     Chem.Kekulize(d,clearAromaticFlags=True)

In [ ]:
# the one just for number of lines - used for the entire drugbank
def rowfunc(smiles):
    # search the smiles in "row" through all the structures in "database"
    substruct = Chem.MolFromSmiles(smiles)
    Chem.SanitizeMol(substruct)
    
    # used to Kekulize, but currently not implemented.
    # SanitizeMol will cause aromatic bonds to be encoded as such.
    
#     Chem.Kekulize(substruct,clearAromaticFlags=True)
    
    substruct_matches = 0
    for database_mol in tqdm(drugs):
        if database_mol.HasSubstructMatch(substruct):
            substruct_matches += 1
    return substruct_matches

In [ ]:

result = process_map(rowfunc, list(data.smiles), chunksize=1000)


In [ ]:
data["drugbank_matches"] = result

In [ ]:
data.sort_values("drugbank_matches")

In [ ]:
# take a look
data.head()

In [ ]:
# save to csv as a backup 
data.to_csv("./data_files/acid_amine_smiles_mindist_dbank.csv")